In [2]:
import numpy as np
from numpy import random
from copy import deepcopy
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import pandas as pd


In [3]:
# general settings used for both generative model and fitting
plenmax=12
nbtype = 3 # in total three types of beat
epsilon = 0.1 # probability of small variation

# generative_model

In [4]:
# pattern generator: uniform for pattern length, uniform for each specific pattern. 
# here includes repetition. like 1010 is the same as 10 but not reduced. This will be dealt with latter
# also it contains like 0000. alternative: force there to be at least one 2. 
def gen_pattern(clock = []):
    
    if len(clock)==0: #no prior
        plen = random.randint(plenmax)+1 # pattern len minimum = 1
        pattern = [random.randint(nbtype) for k in range(plen)]  
        while 2 not in pattern:
            pattern = [random.randint(nbtype) for k in range(plen)]  
    else:
        pattern = [random.randint(nbtype) for k in clock]
        pattern[clock==2] = 2
    return pattern

def ptn_devia(ptn,eps):
    for kp,p in enumerate(ptn):
        if random.rand()<eps:
            ptn[kp] = (p+random.randint(nbtype-1)+1)%nbtype # change p to something else
    return ptn

# repetition controller: transition probability like in markov model
def rep_controller(repnow=0):
    prep = fun_prep(repnow) # higher repetition => less likely to keep repeting. --wonder what's the corresponding prob distribution??
    dorep =  random.rand()<prep
    repnow += dorep
    return dorep
def fun_prep(repnow):
    return 1/(2+repnow)

def meta_controller(metaprior = 'simple',nstep=24):        
    # generate beats by combining the controller and pattern generator
    allbts = []
    allpatterns = []
    # initialize with the first pattern
    pattern = gen_pattern()
    allbts += pattern
    allpatterns.append({'ptn':pattern,'nrep':0})
    if metaprior == 'simple':
        # simplest model: just continue without refering to pervious patterns
        for kstep in range(nstep-1):
            thisptn = allpatterns[-1]        
            dorep = rep_controller(thisptn['nrep'])
            if dorep:
                allbts += ptn_devia(thisptn['ptn'],eps)
                thisptn['nrep'] += 1
            else:
                newptn = gen_pattern()
                allbts += ptn_devia(newptn,eps)
                allpatterns.append({'ptn':newptn,'nrep':0})
                
    elif metaprior == 'VCVC': # verse, chorus, verse, chorus; in chorus the pattern is coherent
        
        verselen = 8
        choruslen = 4
        # generate the first V-C structure
        for kstep in range(verselen-1):
            thisptn = allpatterns[-1]        
            dorep = rep_controller(thisptn['nrep'])
            if dorep:
                allbts += thisptn['ptn']
                thisptn['nrep'] += 1
            else:
                newptn = gen_pattern()
                allbts += newptn
                allpatterns.append({'ptn':newptn,'nrep':0})
        chorusptn = gen_pattern()        
        for k in range(choruslen):
            allbts += chorusptn
        allpatterns.append({'ptn':chorusptn,'nrep':choruslen})
        
        # generate the following, with small probability of changing to new beats
        for k in range(int(nstep/(verselen+choruslen))-1): 
            if 
            allbts += allbts
            allpatterns +=allpatterns
            
    elif metaprior == 'comeback': # when transition to diff pattern, prefer patterns appeared before        
        verselen = 8
        choruslen = 4
        # generate the first V-C structure
        for kstep in range(verselen-1):
            thisptn = allpatterns[-1]        
            dorep = rep_controller(thisptn['nrep'])
            if dorep:
                allbts += thisptn['ptn']
                thisptn['nrep'] += 1
            else:
                newptn = gen_pattern()
                allbts += newptn
                allpatterns.append({'ptn':newptn,'nrep':0})
        chorusptn = gen_pattern()        
        for k in range(choruslen):
            allbts += chorusptn
        allpatterns.append({'ptn':chorusptn,'nrep':choruslen})
        
        # generate the following, with small probability of changing to new beats
        for k in range(int(nstep/(verselen+choruslen))-1): 
            if 
            allbts += allbts
            allpatterns +=allpatterns
        
    return allbts, allpatterns

In [5]:
allbts, allpatterns=meta_controller(metaprior = 'VCVC',nstep=24)

# complexity

In [6]:
def ptn_posterior(ptn,data,clock=[]):
    prior = 1/plenmax/(nbtype**len(ptn) - (nbtype-1)**len(ptn)) # exclude patterns without a stress (2)
    ll=0
    for k,beat in enumerate(data):
        plen = len(ptn)
        ll+=np.log((ptn[k%plen]==beat)*(1-epsilon) +(ptn[k%plen]!=beat)*epsilon )
    return np.log(prior) + ll
#TODO: with metaprior, transition will have structural bias...to be figured out
def stim_posterior(ptns,data,metaprior='simple',clock=[]):
    logpstr=0
    idx=0
    for ptn in ptns:
        nrep = ptn['nrep']
        for k in range(nrep):
            logpstr += np.log(fun_prep(k)) # prob of repeating
        if ptn != ptns[-1]:
            logpstr += np.log(1-fun_prep(nrep)) # prob of transitioning
        # get the pattern posterior for the corresponding chunk
        ptnlen=len(ptn['ptn'])*ptn['nrep']
        chunk = data[idx:idx+ptnlen]
        idx=idx+ptnlen
        logpstr += ptn_posterior(ptn['ptn'],chunk)
    return logpstr

In [7]:
# for convenience: generate actual beats from pattern 
def ptns2beat(ptns):
    allbts=[]
    for ptn in ptns:
        for krep in range(ptn['nrep']):
            allbts += ptn_devia(ptn['ptn'],epsilon)
    return allbts
            

In [13]:
# get posterior for the pattern group
# sanity check 1: shorter patterns are preferred? -- yes if only see pattern posterior; not always given the transition function
ptngrp = [{'ptn':[2,1,0,1],'nrep':4}]
beats=ptns2beat(ptngrp)
print(stim_posterior(ptngrp,beats))
ptngrp = [{'ptn':[2,1,0,1,2,1,0,1],'nrep':2}]
beats=ptns2beat(ptngrp)
print(stim_posterior(ptngrp,beats))

-13.132553913
-16.9087571953


# fitting

In [ ]:

def ptn_hypothesis(beats):
    ptn = beats
    